In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
#import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  


import sys

import plotly.plotly as py
import plotly.graph_objs as go


#i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
import plotly.tools as tls
tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


#import pygraphviz
#from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



# ##### for getting geolocation data  and to calculate distance between two geolocations
# import requests
# import json
#import geopy.distance   


In [2]:


####### dict folder: list of users
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_list_folders_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_list_folders = pickle.load(handle)
#df_users = pd.DataFrame.from_dict(dict_folder_id_list_users,orient='index')
print len(dict_user_id_list_folders)



####### dict folder: list of users
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_list_users_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_folder_id_list_users = pickle.load(handle)
#df_users = pd.DataFrame.from_dict(dict_folder_id_list_users,orient='index')
print len(dict_folder_id_list_users)





########  csv  users' attr
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'DROPBOX_basic_user_attr_COMPLETE.csv'
df_users_attr = pd.read_csv(path+input_file, sep=';',na_values=["NAN","NaN","nan","-1","null",""]) 
df_users_attr.drop('Unnamed: 0', axis=1, inplace=True)
print "num. users:",len(df_users_attr.user_id.unique()),df_users_attr.shape    # 400000   (5992006, 7)







####### dict folder_id folder attr  (dict and df)
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat.pickle'
#pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr.pickle'
with open(pickle_name, 'rb') as handle:
    dict_folder_id_folder_attr = pickle.load(handle)
#df_users = pd.DataFrame.from_dict(dict_folder_id_list_users,orient='index')
print len(dict_folder_id_folder_attr)




csv_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/folder_id_folder_attr_with_time_attr.csv'
df_folder_id_folder_attr = pd.read_csv(csv_name, sep=';',na_values=["NAN","NaN","nan","-1","null",""]) 
print "num. folders:",df_folder_id_folder_attr.shape







438284
519045
519045


In [78]:
list_folders_added_so_far=[]
for folder_id in tqdm_notebook(dict_folder_id_list_users):
    #print "folder:",folder_id
    list_users=dict_folder_id_list_users[folder_id]
    #print "  users in it:", list_users
    df_selected_users=df_users_attr[df_users_attr['user_id'].isin(list_users)]
    
    
    
    
    
    
    mean_category_total_publ=df_selected_users.category_total_publ.mean()   # by default, it ignore any NaN values to  compute the avg
    #print "    avg cat. num. publ:",mean_category_total_publ
    
    mean_category_total_last_auth=df_selected_users.category_total_last_auth.mean()
    #print "    avg cat. num. publ last author:",mean_category_total_last_auth
    
    mean_category_total_cit=df_selected_users.category_total_cit.mean()
    #print "    avg cat. num. cit:",mean_category_total_cit
    
    
    
    dict_folder_id_folder_attr[folder_id]["avg_categ_num_publ"] = mean_category_total_publ
    dict_folder_id_folder_attr[folder_id]["avg_categ_num_last_auth"] = mean_category_total_last_auth
    dict_folder_id_folder_attr[folder_id]["avg_categ_num_cit"] = mean_category_total_cit
    
    
    list_folders_added_so_far.append(folder_id)
    
    
    
    
print "done adding the new avg variables to dict_folder_id_folder_attr"    
    

print 

pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_folder_id_folder_attr, handle)
print "written:", pickle_name




csv_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat.csv'
df_from_dict = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')
## add foler_id from index
df_from_dict.to_csv(csv_name, sep=';')
print "written:", csv_name




done adding the new avg variables to dict_folder_id_folder_attr

written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat.pickle
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat.csv


In [83]:
df_from_dict.head()

,frac_simult_days_over_active_days,most_common_categ_num_publ,num_unique_projects_members_work_on,num_fields,avg_geo_dist_km,most_common_categ_num_last_auth,folder_univ_median_ranking,eff_num_members_by_days_present,num_above_avg_contributors,gini_num_days_present_over_members,...,folder_tot_num_edits,std_geo_dist_km,fraction_SR,most_common_num_cit,number_active_members,overall_num_active_days,folder_tot_num_adds,num_time_stamps,folder_tot_num_dels,folder_id
551107.0,0.0,NaN,13,1.0,291.818436,NaN,146.0,1.0,NaN,0.500000,...,6.0,0.0,NaN,NaN,2,2,5.0,2,0.0,551107.0
574131.0,NaN,NaN,116,1.0,0.000000,NaN,7.0,0.0,NaN,NaN,...,0.0,0.0,NaN,NaN,1,0,0.0,0,0.0,574131.0
582764.0,0.0,NaN,36,1.0,0.000000,NaN,10.0,1.0,NaN,0.666667,...,3.0,0.0,NaN,NaN,3,3,3.0,3,0.0,582764.0
586551.0,0.0,4.0,8,1.0,0.000000,1.0,16.0,1.0,NaN,0.000000,...,4.0,0.0,1.0,4.0,1,103,518.0,103,1453.0,586551.0
590047.0,0.0,5.0,64,1.0,0.000000,4.0,1.0,1.0,NaN,0.500000,...,0.0,0.0,1.0,5.0,2,1,4.0,1,0.0,590047.0


In [84]:
len(df_from_dict.folder_id.unique())

519045

In [85]:
new_dict_user_id_world_ranking=area_dict = dict(zip(df_users_attr['user_id'], df_users_attr['world_ranking']))

list_tops=[10.,20.,30.,40.]

for top in list_tops:
    print "top:", top
    list_folders_to_include_in_top=[]


    for folder_id in dict_folder_id_list_users:
        list_users_in_folder=dict_folder_id_list_users[folder_id]

        for user_id in list_users_in_folder:
            if new_dict_user_id_world_ranking[user_id]<= top:
                list_folders_to_include_in_top.append(folder_id)


    print "total number folders",len(dict_folder_id_list_users) ,       
    list_folders_to_include_in_top =   list(set(list_folders_to_include_in_top))         
    print "  number of selected folders",len(list_folders_to_include_in_top)





    ##### i make the selection of the df by the corresponding folders    
    df_selected_folders=df_from_dict[df_from_dict['folder_id'].isin(list_folders_to_include_in_top)]



    new_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat_top_'+str(int(top))+".csv"
    df_selected_folders.to_csv(new_name, sep=';', index=False)   # index=false so i dont write out the column corresponding to the index 
    print "written scv for selected folders:", new_name
    print df_selected_folders.shape
    print 
    print 

top: 10.0
total number folders 519045   number of selected folders 88796
written scv for selected folders: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat_top_10.csv
(88796, 43)


top: 20.0
total number folders 519045   number of selected folders 154819
written scv for selected folders: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat_top_20.csv
(154819, 43)


top: 30.0
total number folders 519045   number of selected folders 210178
written scv for selected folders: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat_top_30.csv
(210178, 43)


top: 40.0
total number folders 519045   number of selected folders 230514
written scv for selected folders: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_added_avg_cat_top_

In [86]:
sorted(df_selected_folders.columns)

['act_per_day',
 'act_per_day_per_active_user',
 'act_period',
 'avg_categ_num_cit',
 'avg_categ_num_last_auth',
 'avg_categ_num_publ',
 'avg_geo_dist_km',
 'avg_interevent_time',
 'burstiness',
 'dominated_folder',
 'eff_num_members',
 'eff_num_members_by_days_present',
 'folder_activity_GINI',
 'folder_dominator',
 'folder_id',
 'folder_lifespan',
 'folder_tot_act',
 'folder_tot_num_adds',
 'folder_tot_num_dels',
 'folder_tot_num_edits',
 'folder_univ_SD_ranking',
 'folder_univ_mean_ranking',
 'folder_univ_median_ranking',
 'frac_simult_days_over_active_days',
 'frac_simult_days_overall',
 'fract_above_avg_contributors',
 'fraction_SR',
 'fraction_work_by_SR',
 'gini_num_days_present_over_members',
 'most_common_career_stage',
 'most_common_categ_num_last_auth',
 'most_common_categ_num_publ',
 'most_common_num_cit',
 'most_common_univ_ranking',
 'num_above_avg_contributors',
 'num_countries',
 'num_fields',
 'num_time_stamps',
 'num_unique_projects_members_work_on',
 'num_universitie

In [65]:
for folder_id in list_folders_added_so_far:
    if len(dict_folder_id_list_users[folder_id])>3:
        if dict_folder_id_folder_attr[folder_id]['avg_categ_num_cit'] >=1.:
            print folder_id

659554317.0
332223832.0
164626470.0
207618090.0
160432180.0
950009931.0
34777784.0
156237914.0
654311516.0
930087008.0
508559480.0
554696838.0
896357752.0
90177748.0
803176490.0
244318456.0
888144151.0
910164252.0
1022361891.0
1267728682.0
843055404.0
278921519.0
732430392.0
1005584770.0
1197474182.0
978321816.0
865075619.0
1324526324.0
1326449088.0
509496657.0
914358795.0


In [70]:
folder_id=914358795.0
list_users=dict_folder_id_list_users[folder_id]
df_selected_users=df_users_attr[df_users_attr['user_id'].isin(list_users)]
df_selected_users[["user_id","world_ranking","category_total_publ","category_total_last_auth","category_total_cit"]]

,user_id,world_ranking,category_total_publ,category_total_last_auth,category_total_cit
120531,74518177.0,NaN,2.0,1.0,3.0
120532,139864851.0,NaN,NaN,NaN,NaN
120533,139864538.0,NaN,NaN,NaN,NaN
120534,103220043.0,NaN,NaN,NaN,NaN


In [71]:
dict_folder_id_folder_attr[folder_id]

{'act_per_day': 1.0,
 'act_per_day_per_active_user': 0.25,
 'act_period': 1,
 'avg_categ_num_cit': 3.0,
 'avg_categ_num_last_auth': 1.0,
 'avg_categ_num_publ': 2.0,
 'avg_geo_dist_km': 0.0,
 'avg_interevent_time': nan,
 'burstiness': nan,
 'dominated_folder': 1,
 'eff_num_members': 1.0,
 'eff_num_members_by_days_present': 1.0,
 'folder_activity_GINI': 0.75,
 'folder_dominator': 1.0,
 'folder_lifespan': 79,
 'folder_tot_act': 1.0,
 'folder_tot_num_adds': 0.0,
 'folder_tot_num_dels': 0.0,
 'folder_tot_num_edits': 1.0,
 'folder_univ_SD_ranking': nan,
 'folder_univ_mean_ranking': nan,
 'folder_univ_median_ranking': nan,
 'frac_simult_days_over_active_days': 0.0,
 'frac_simult_days_overall': 0.0,
 'fract_above_avg_contributors': nan,
 'fraction_SR': 0.0,
 'fraction_work_by_SR': nan,
 'gini_num_days_present_over_members': 0.75,
 'most_common_career_stage': 'jr',
 'most_common_categ_num_last_auth': 1.0,
 'most_common_categ_num_publ': 2.0,
 'most_common_num_cit': 3.0,
 'most_common_univ_rankin

In [3]:
####### dict folder: list of users
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_new.pickle'
with open(pickle_name, 'rb') as handle:
    G = pickle.load(handle)
#df_users = pd.DataFrame.from_dict(dict_folder_id_list_users,orient='index')
print len(G.nodes()), len(G.edges())  # 438,284       3,795,947


438284


In [7]:
# i add a new index to cluster together forlders that share at least one user  (so technically, the attributes of those folders are not really independent)



cont_overlapping=0
dict_folder_id_new_index_overlapping_members={}
dict_new_index_overlap_list_folders={}

#

for user_id in tqdm_notebook(dict_user_id_list_folders):
    
 
    
    lista_folders=dict_user_id_list_folders[user_id]
    
    
    
    if len(lista_folders) > 1:  # if a user works in two different teams
        
        
        lista_pares_folders=itertools.combinations(lista_folders, 2)

        for pair in lista_pares_folders:
            
            folder1=pair[0]
            folder2=pair[1]
        
        
        try:
            dict_folder_id_new_index_overlapping_members[folder1].append(cont_overlapping)
        except KeyError:
            dict_folder_id_new_index_overlapping_members[folder1]=[]
            dict_folder_id_new_index_overlapping_members[folder1].append(cont_overlapping)
        

        
        try:
            dict_folder_id_new_index_overlapping_members[folder2].append(cont_overlapping)
        except KeyError:
            dict_folder_id_new_index_overlapping_members[folder2]=[]
            dict_folder_id_new_index_overlapping_members[folder2].append(cont_overlapping)
            
            
            
            
            
            
        dict_new_index_overlap_list_folders[cont_overlapping]=(folder1, folder2)            
            
        cont_overlapping +=1
        
print "done."
      



done.


In [10]:
len(dict_folder_id_new_index_overlapping_members)


213884